## 📦 1. Configuration de l'environnement

In [1]:
# Vérifier si on est sur Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✓ Exécution sur Google Colab")
else:
    print("✓ Exécution en local")

✓ Exécution sur Google Colab


In [2]:
# Installation des dépendances sur Colab
if IN_COLAB:
    !pip install networkx==3.2.1 pandas matplotlib seaborn scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.2 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.6.1
    Uninstalling networkx-3.6.1:
      Successfully uninstalled networkx-3.6.1


## 📂 2. Clonage du dépôt GitHub

In [3]:
import os

if IN_COLAB:
    # Cloner le repo si pas déjà fait
    if not os.path.exists('/content/NSGL-Projet'):
        !git clone https://github.com/tanguycesar/NSGL-Projet.git
        %cd /content/NSGL-Projet
    else:
        %cd /content/NSGL-Projet
        !git pull

    print(f"📁 Dossier de travail: {os.getcwd()}")
    print(f"📄 Fichiers Python disponibles:")
    !ls -l *.py

Cloning into 'NSGL-Projet'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: '/content/NSGL-Projet'
/content
📁 Dossier de travail: /content
📄 Fichiers Python disponibles:
ls: cannot access '*.py': No such file or directory


## 🌐 3. Téléchargement des données Facebook100

In [ ]:
import urllib.request
import zipfile

data_dir = 'facebook100' if not IN_COLAB else '/content/NSGL-Projet/facebook100'

# Télécharger si pas déjà présent
if not os.path.exists(data_dir) or len([f for f in os.listdir(data_dir) if f.endswith('.gml')]) == 0:
    print("📥 Téléchargement des données Facebook100...")

    os.makedirs(data_dir, exist_ok=True)

    url = 'https://gitlab.imt-atlantique.fr/boucheddf/projects-tried/-/raw/main/facebook100.zip'
    zip_path = os.path.join(data_dir, 'facebook100.zip')

    urllib.request.urlretrieve(url, zip_path)
    print(f"✓ Téléchargement terminé: {zip_path}")

    # Extraire
    print("📦 Extraction...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)

    os.remove(zip_path)
    print("✓ Extraction terminée")
else:
    print("✓ Données Facebook100 déjà présentes")

# Lister les réseaux
gml_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.gml')])
print(f"\n📊 {len(gml_files)} réseaux disponibles")
print("Exemples:", gml_files[:5])

## 🔧 4. Imports et configuration GPU

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error
import torch
from collections import defaultdict
import random
import warnings
warnings.filterwarnings('ignore')

# Configuration graphiques
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Configuration GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️  Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Mémoire: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

---
# Question 4: Link Prediction
## Prédiction de liens avec métriques manuelles

### 4.1 Implémentation des métriques

In [ ]:
def common_neighbors_score(G, u, v, neighbors_dict):
    """Common Neighbors: |Γ(u) ∩ Γ(v)|"""
    return len(neighbors_dict[u] & neighbors_dict[v])

def jaccard_score(G, u, v, neighbors_dict):
    """Jaccard: |Γ(u) ∩ Γ(v)| / |Γ(u) ∪ Γ(v)|"""
    intersection = len(neighbors_dict[u] & neighbors_dict[v])
    union = len(neighbors_dict[u] | neighbors_dict[v])
    return intersection / union if union > 0 else 0

def adamic_adar_score(G, u, v, neighbors_dict):
    """Adamic-Adar: Σ_{w ∈ Γ(u) ∩ Γ(v)} 1 / log(|Γ(w)|)"""
    common = neighbors_dict[u] & neighbors_dict[v]
    score = 0
    for w in common:
        degree_w = len(neighbors_dict[w])
        if degree_w > 1:
            score += 1.0 / np.log(degree_w)
    return score

### 4.2 Génération des paires candidates (optimisé)

In [ ]:
def get_candidate_pairs(G, max_candidates=50000):
    """
    Génère des paires candidates en limitant à max_candidates.
    Optimisations:
    - Limite de 50k paires (suffisant pour k=400)
    - Échantillonnage pour nœuds > 100 voisins
    - Vérification O(1) avec set
    """
    edges_set = set(G.edges())
    candidates = []

    nodes = list(G.nodes())
    random.shuffle(nodes)  # Diversité

    neighbors_dict = {n: set(G.neighbors(n)) for n in G.nodes()}

    for u in nodes:
        if len(candidates) >= max_candidates:
            break

        neighbors_u = neighbors_dict[u]

        # Voisins de voisins
        neighbors_of_neighbors = set()
        for neighbor in neighbors_u:
            neighbor_neighbors = neighbors_dict[neighbor]

            # Échantillonner si trop de voisins
            if len(neighbor_neighbors) > 100:
                neighbor_neighbors = set(random.sample(list(neighbor_neighbors), 100))

            neighbors_of_neighbors.update(neighbor_neighbors)

        # Paires non connectées
        for v in neighbors_of_neighbors:
            if v != u and v not in neighbors_u:
                edge = tuple(sorted([u, v]))
                if edge not in edges_set:
                    candidates.append((u, v))

                    if len(candidates) >= max_candidates:
                        break

    return candidates, neighbors_dict

### 4.3 Fonction d'évaluation

In [ ]:
def evaluate_predictor(G, predictor_name, f_values, k_values, max_candidates=50000):
    """
    Évalue un prédicteur sur différentes fractions et valeurs de k
    """
    results = []

    for f in f_values:
        print(f"  📊 Fraction {f:.0%}...", end=" ")

        # Supprimer f% des arêtes
        edges = list(G.edges())
        num_remove = int(len(edges) * f)
        removed_edges = random.sample(edges, num_remove)

        G_train = G.copy()
        G_train.remove_edges_from(removed_edges)
        removed_set = set(tuple(sorted(e)) for e in removed_edges)

        # Générer candidates
        candidates, neighbors_dict = get_candidate_pairs(G_train, max_candidates)

        # Calculer scores
        scores = []
        for u, v in candidates:
            if predictor_name == "CommonNeighbors":
                score = common_neighbors_score(G_train, u, v, neighbors_dict)
            elif predictor_name == "Jaccard":
                score = jaccard_score(G_train, u, v, neighbors_dict)
            elif predictor_name == "AdamicAdar":
                score = adamic_adar_score(G_train, u, v, neighbors_dict)
            scores.append((u, v, score))

        # Trier par score décroissant
        scores.sort(key=lambda x: x[2], reverse=True)

        # Évaluer pour chaque k
        for k in k_values:
            top_k = scores[:k]
            top_k_pairs = [tuple(sorted([u, v])) for u, v, _ in top_k]

            # Précision et Rappel
            hits = sum(1 for pair in top_k_pairs if pair in removed_set)
            precision = hits / k if k > 0 else 0
            recall = hits / len(removed_set) if len(removed_set) > 0 else 0

            results.append({
                'predictor': predictor_name,
                'f': f,
                'k': k,
                'precision': precision,
                'recall': recall
            })

        print("✓")

    return results

### 4.4 Exécution sur 15 réseaux

In [ ]:
# Sélection de 15 réseaux (moyens et grands)
selected_networks_q4 = [
    'Caltech36.gml', 'Reed98.gml', 'Haverford76.gml', 'Swarthmore42.gml', 'Amherst41.gml',
    'Hamilton46.gml', 'Colgate88.gml', 'Oberlin44.gml', 'Middlebury45.gml', 'Wellesley22.gml',
    'Brandeis99.gml', 'Lehigh96.gml', 'American75.gml', 'Bucknell39.gml', 'Duke14.gml'
]

# Paramètres
f_values = [0.05, 0.1, 0.15, 0.2]
k_values = [50, 100, 200, 300, 400]
predictors = ["CommonNeighbors", "Jaccard", "AdamicAdar"]

all_results_q4 = []

print("🚀 Début de l'analyse Question 4...\n")

for i, network_file in enumerate(selected_networks_q4, 1):
    print(f"📊 [{i}/15] {network_file}")

    network_path = os.path.join(data_dir, network_file)
    G = nx.read_gml(network_path)
    G = G.to_undirected()

    print(f"   Nœuds: {G.number_of_nodes()}, Arêtes: {G.number_of_edges()}")

    for predictor in predictors:
        print(f"  🔍 {predictor}")
        results = evaluate_predictor(G, predictor, f_values, k_values)

        for r in results:
            r['network'] = network_file.replace('.gml', '')
        all_results_q4.extend(results)

    print()

# Sauvegarder résultats
df_q4 = pd.DataFrame(all_results_q4)
df_q4.to_csv('question4_link_prediction_results_all.csv', index=False)
print("💾 Résultats sauvegardés: question4_link_prediction_results_all.csv")

### 4.5 Visualisation des résultats agrégés

In [ ]:
# Agréger par prédictor, f, k
df_q4_agg = df_q4.groupby(['predictor', 'f', 'k']).agg({
    'precision': ['mean', 'std'],
    'recall': ['mean', 'std']
}).reset_index()

df_q4_agg.columns = ['predictor', 'f', 'k', 'precision_mean', 'precision_std', 'recall_mean', 'recall_std']

# Graphiques
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Question 4: Link Prediction - Résultats Agrégés (15 réseaux)', fontsize=16, fontweight='bold')

for i, f in enumerate(f_values):
    ax = axes[i // 2, i % 2]
    data_f = df_q4_agg[df_q4_agg['f'] == f]

    for predictor in predictors:
        data_p = data_f[data_f['predictor'] == predictor]
        ax.errorbar(data_p['k'], data_p['precision_mean'], yerr=data_p['precision_std'],
                    label=predictor, marker='o', capsize=5, linewidth=2)

    ax.set_title(f'Fraction f = {f:.0%}', fontsize=14, fontweight='bold')
    ax.set_xlabel('k (nombre de prédictions)', fontsize=12)
    ax.set_ylabel('Précision moyenne', fontsize=12)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('question4_link_prediction_aggregated.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📈 Graphique sauvegardé: question4_link_prediction_aggregated.png")

---
# Question 5: Label Propagation
## Propagation de labels avec GPU

### 5.1 Implémentation avec PyTorch

In [ ]:
def label_propagation_gpu(G, attribute_name, fraction_missing, device='cuda', max_iterations=200):
    """
    Propagation de labels avec PyTorch et matrices sparses GPU
    """
    # Extraire labels
    nodes = list(G.nodes())
    labels = [G.nodes[n].get(attribute_name, None) for n in nodes]

    # Filtrer valeurs manquantes
    valid_indices = [i for i, label in enumerate(labels) if label is not None and label != 0]
    if len(valid_indices) == 0:
        return None

    nodes = [nodes[i] for i in valid_indices]
    labels = [labels[i] for i in valid_indices]

    # Sous-graphe
    G_sub = G.subgraph(nodes).copy()
    if G_sub.number_of_nodes() == 0:
        return None

    # Mapping nœuds -> indices
    node_to_idx = {node: idx for idx, node in enumerate(G_sub.nodes())}
    labels_array = np.array([labels[i] for i in range(len(nodes))])

    # Marquer fraction_missing comme inconnus
    n = len(labels_array)
    num_remove = int(n * fraction_missing)
    indices_to_remove = np.random.choice(n, size=num_remove, replace=False)

    known_mask = np.ones(n, dtype=bool)
    known_mask[indices_to_remove] = False

    true_labels = labels_array.copy()
    labels_array[~known_mask] = 0  # Inconnu

    # Construire matrice d'adjacence sparse
    edges = list(G_sub.edges())
    edge_index = torch.tensor(
        [[node_to_idx[u], node_to_idx[v]] for u, v in edges] +
        [[node_to_idx[v], node_to_idx[u]] for u, v in edges],
        dtype=torch.long
    ).t().to(device)

    # Normalisation par degré
    degrees = torch.bincount(edge_index[0], minlength=n).float().to(device)
    degrees[degrees == 0] = 1  # Éviter division par 0
    edge_weight = 1.0 / degrees[edge_index[0]]

    # Labels sur GPU
    labels_tensor = torch.tensor(labels_array, dtype=torch.float32).to(device)
    known_mask_tensor = torch.tensor(known_mask, dtype=torch.bool).to(device)

    # Propagation itérative
    for iteration in range(max_iterations):
        new_labels = torch.zeros_like(labels_tensor)
        new_labels.index_add_(0, edge_index[1], labels_tensor[edge_index[0]] * edge_weight)

        # Conserver labels connus
        labels_tensor = torch.where(known_mask_tensor, labels_tensor, new_labels)

        # Convergence
        if iteration > 0 and torch.allclose(labels_tensor, new_labels, atol=1e-4):
            break

    # Prédictions
    predicted_labels = labels_tensor.cpu().numpy()
    predicted_labels = np.round(predicted_labels).astype(int)

    # Évaluation
    y_true = true_labels[~known_mask]
    y_pred = predicted_labels[~known_mask]

    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    mae = mean_absolute_error(y_true, y_pred)

    return {
        'accuracy': accuracy,
        'f1_score': f1,
        'mae': mae,
        'num_nodes': n,
        'num_missing': num_remove
    }

### 5.2 Exécution sur 15 réseaux

In [ ]:
# Mêmes 15 réseaux
selected_networks_q5 = selected_networks_q4

# Paramètres
attributes = ['dorm', 'major_index', 'gender']
fractions = [0.1, 0.2, 0.3]

all_results_q5 = []

print("🚀 Début de l'analyse Question 5...\n")

for i, network_file in enumerate(selected_networks_q5, 1):
    print(f"📊 [{i}/15] {network_file}")

    network_path = os.path.join(data_dir, network_file)
    G = nx.read_gml(network_path)
    G = G.to_undirected()

    for attr in attributes:
        print(f"  🏷️  Attribut: {attr}")

        for frac in fractions:
            print(f"    📉 Fraction: {frac:.0%}", end=" ")

            result = label_propagation_gpu(G, attr, frac, device=device)

            if result:
                result.update({
                    'network': network_file.replace('.gml', ''),
                    'attribute': attr,
                    'fraction': frac
                })
                all_results_q5.append(result)
                print(f"✓ Acc={result['accuracy']:.3f}")
            else:
                print("⚠️ Pas de données valides")

    print()

# Sauvegarder
df_q5 = pd.DataFrame(all_results_q5)
df_q5.to_csv('question5_label_propagation_detailed.csv', index=False)
print("💾 Résultats sauvegardés: question5_label_propagation_detailed.csv")

### 5.3 Visualisation agrégée

In [ ]:
# Agréger par attribut et fraction
df_q5_agg = df_q5.groupby(['attribute', 'fraction']).agg({
    'accuracy': ['mean', 'std'],
    'f1_score': ['mean', 'std'],
    'mae': ['mean', 'std']
}).reset_index()

df_q5_agg.columns = ['attribute', 'fraction', 'acc_mean', 'acc_std', 'f1_mean', 'f1_std', 'mae_mean', 'mae_std']

# Graphiques
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Question 5: Label Propagation - Résultats Agrégés (15 réseaux)', fontsize=16, fontweight='bold')

metrics = [('acc_mean', 'acc_std', 'Accuracy'),
           ('f1_mean', 'f1_std', 'F1-Score'),
           ('mae_mean', 'mae_std', 'MAE')]

for i, (mean_col, std_col, title) in enumerate(metrics):
    ax = axes[i]

    for attr in attributes:
        data_attr = df_q5_agg[df_q5_agg['attribute'] == attr]
        ax.errorbar(data_attr['fraction'], data_attr[mean_col], yerr=data_attr[std_col],
                    label=attr, marker='o', capsize=5, linewidth=2)

    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel('Fraction de labels manquants', fontsize=12)
    ax.set_ylabel(f'{title} moyen', fontsize=12)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('question5_label_propagation_aggregated.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📈 Graphique sauvegardé: question5_label_propagation_aggregated.png")

## 📦 6. Export des résultats

In [ ]:
import zipfile

# Créer archive ZIP
zip_filename = 'results_q4_q5.zip'
files_to_zip = [
    'question4_link_prediction_results_all.csv',
    'question4_link_prediction_aggregated.png',
    'question5_label_propagation_detailed.csv',
    'question5_label_propagation_aggregated.png'
]

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files_to_zip:
        if os.path.exists(file):
            zipf.write(file)
            print(f"✓ Ajouté: {file}")

print(f"\n📦 Archive créée: {zip_filename}")

if IN_COLAB:
    from google.colab import files
    files.download(zip_filename)
    print("📥 Téléchargement lancé!")

## ✅ Résumé

In [ ]:
print("="*60)
print("📊 RÉSUMÉ DE L'ANALYSE")
print("="*60)
print(f"\n🔗 QUESTION 4 - Link Prediction:")
print(f"  • Réseaux analysés: {len(selected_networks_q4)}")
print(f"  • Prédicteurs: {', '.join(predictors)}")
print(f"  • Fractions: {f_values}")
print(f"  • Valeurs de k: {k_values}")
print(f"  • Total expériences: {len(df_q4)}")

print(f"\n🏷️  QUESTION 5 - Label Propagation:")
print(f"  • Réseaux analysés: {len(selected_networks_q5)}")
print(f"  • Attributs: {', '.join(attributes)}")
print(f"  • Fractions: {fractions}")
print(f"  • Device: {device}")
print(f"  • Total expériences: {len(df_q5)}")

print(f"\n📁 Fichiers générés:")
for file in files_to_zip:
    if os.path.exists(file):
        size_mb = os.path.getsize(file) / 1024 / 1024
        print(f"  ✓ {file} ({size_mb:.2f} MB)")

print("\n" + "="*60)
print("✅ ANALYSE TERMINÉE")
print("="*60)